# Create an interface with strain matching

Use Zur and McGill superlattices matching [algorithm](https://doi.org/10.1063/1.3330840) to select an interface between two materials with minimal strain.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Set slab parameters for the substrate and film in cell 2.1. (or use default).
1. Set interface parameters in cell 3.1. (or use default).
1. Click “Run” > “Run All” to run all cells. 
1. Wait for the run to complete (depending on the parameters can take a few min). 
1. Scroll down to view results. 

## Summary
1. Prepare the Environment: Set up the notebook and install packages, preview the input materials
1. Create substrate and film slabs and select the terminations
1. Generate interfaces with strain matcher and plot strain vs number of atoms 
1. Select the interface with the desired strain and visualize it

## Notes
1. We perform strain matching on the slabs to extract the supercell dimensions. The algorithm has a set of parameters, such as the maximum area considered.
1. When the strain matching is finished, the interface with the lowest strain (and the smallest number of atoms) is selected. 
1. ZSL strain matching is performed using Pymatgen [implementation](https://pymatgen.org/pymatgen.analysis.interfaces.html#pymatgen.analysis.interfaces.zsl).
1. For more information, see [Introduction](Introduction.ipynb)
<!-- # TODO: use a hashtag-based anchor link to interface creation documention above -->


## 1. Prepare the Environment
### 1.1. Set up the notebook 

Set the following flags to control the notebook behavior 

In [ ]:
# Enable interactive selection of terminations via UI prompt
IS_TERMINATIONS_SELECTION_INTERACTIVE = False 

FILM_INDEX = 1 # Index in the list of materials, to access as materials[FILM_INDEX]
FILM_MILLER_INDICES = (0,0,1)
FILM_THICKNESS = 1  # in atomic layers
FILM_VACUUM = 0.0  # in angstroms
FILM_XY_SUPERCELL_MATRIX = [[1, 0], [0, 1]]
FILM_USE_ORTHOGONAL_Z = True

SUBSTRATE_INDEX = 0
SUBSTRATE_MILLER_INDICES = (0,0,1)
SUBSTRATE_THICKNESS = 3  # in atomic layers
SUBSTRATE_VACUUM = 0.0  # in angstroms
SUBSTRATE_XY_SUPERCELL_MATRIX = [[1, 0], [0, 1]]
SUBSTRATE_USE_ORTHOGONAL_Z = True

# Maximum area for the superlattice search algorithm
MAX_AREA = 50 # in Angstrom^2
# Additional fine-tuning parameters (increase values to get more strained matches):
MAX_AREA_TOLERANCE = 0.09 # in Angstrom^2
MAX_ANGLE_TOLERANCE = 0.03
MAX_LENGTH_TOLERANCE = 0.03
# Set the termination pair indices
TERMINATION_PAIR_INDEX = 0 # Will be overridden in interactive selection is used
INTERFACE_DISTANCE = 3.0  # in Angstrom
INTERFACE_VACUUM = 20.0  # in Angstrom

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip
  
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("create_interface_with_min_strain_zsl.ipynb")

### 1.3. Get input materials and assign `substrate` and `film`
Materials are loaded with `get_data()`. The first material is assigned as substrate and the second as film.

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())
substrate = materials[SUBSTRATE_INDEX]
try: 
    film = materials[FILM_INDEX]
except IndexError:
    print("Film material not found. Re-using substrate material as film.")
    film = substrate

### 1.4. Preview Substrate and Film

In [ ]:
from utils.visualize import visualize_materials as visualize
visualize([substrate, film], repetitions=[3, 3, 1], rotation="0x")

## 2. Configure slabs and select termination pair

### 2.1. Create Substrate and Layer Slabs
Slab Configuration lets define the slab thickness, vacuum, and the Miller indices of the interfacial plane and get the slabs with possible terminations.
Define the substrate slab cell that will be used as a base for the interface and the film slab cell that will be placed on top of the substrate slab.

In [ ]:
from mat3ra.made.tools.build.slab import SlabConfiguration, get_terminations, create_slab

film_slab_configuration = SlabConfiguration(
    bulk=film,
    miller_indices=FILM_MILLER_INDICES,
    thickness=FILM_THICKNESS, # in atomic layers
    vacuum=FILM_VACUUM, # in angstroms
    xy_supercell_matrix=FILM_XY_SUPERCELL_MATRIX,
    use_orthogonal_z=FILM_USE_ORTHOGONAL_Z
)

substrate_slab_configuration = SlabConfiguration(
    bulk=substrate,
    miller_indices=SUBSTRATE_MILLER_INDICES,
    thickness=SUBSTRATE_THICKNESS, # in atomic layers
    vacuum=SUBSTRATE_VACUUM, # in angstroms
    xy_supercell_matrix=SUBSTRATE_XY_SUPERCELL_MATRIX,
    use_orthogonal_z=SUBSTRATE_USE_ORTHOGONAL_Z
)

### 2.2. Get possible terminations for the slabs

In [ ]:
film_slab_terminations = get_terminations(film_slab_configuration)
substrate_slab_terminations = get_terminations(substrate_slab_configuration)

### 2.3. Visualize slabs for all possible terminations

In [ ]:
film_slabs = [create_slab(film_slab_configuration, termination) for termination in film_slab_terminations]
substrate_slabs = [create_slab(substrate_slab_configuration, termination) for termination in substrate_slab_terminations]

visualize([{"material":slab, "title": slab.metadata["build"]["termination"]} for slab in film_slabs ], repetitions=[3, 3, 1], rotation="-90x")
visualize([{"material":slab, "title": slab.metadata["build"]["termination"]} for slab in substrate_slabs ], repetitions=[3, 3, 1], rotation="-90x")  

### 2.4. Print terminations for the interface

In [ ]:
from itertools import product

termination_pairs = list(product(film_slab_terminations, substrate_slab_terminations))    
print("Termination Pairs (Film, Substrate)")
for idx, termination_pair in enumerate(termination_pairs):
    print(f"    {idx}: {termination_pair}")

### 2.5. Select termination pair for the interface

In [ ]:
from utils.io import ui_prompt_select_array_element_by_index, ui_prompt_select_array_element_by_index_pyodide

termination_pair_index = TERMINATION_PAIR_INDEX

termination_pair = termination_pairs[termination_pair_index]
if IS_TERMINATIONS_SELECTION_INTERACTIVE:
    if sys.platform == "emscripten":
        termination_pair = await ui_prompt_select_array_element_by_index_pyodide(termination_pairs, element_name="film/substrate termination pair")
    else:
        termination_pair = ui_prompt_select_array_element_by_index(termination_pairs, element_name="film/substrate termination pair")

## 3. Create interfaces

### 3.1. Initialize the Interface Configuration

In [ ]:
from mat3ra.made.tools.build.interface import InterfaceConfiguration

film_termination, substrate_termination = termination_pair
interface_configuration = InterfaceConfiguration(
    film_configuration=film_slab_configuration,
    substrate_configuration=substrate_slab_configuration,
    film_termination=film_termination,
    substrate_termination=substrate_termination,
    distance=INTERFACE_DISTANCE,
    vacuum= INTERFACE_VACUUM
)

### 3.2. Set Strain Matching Algorithm Parameters (Optional)
The search algorithm for supercells matching can be tuned by setting its parameters directly, otherwise the default values are used.

In [ ]:
from mat3ra.made.tools.build.interface import ZSLStrainMatchingParameters
zsl_strain_matching_parameters = ZSLStrainMatchingParameters(
    max_area=MAX_AREA,
    max_area_tol=MAX_AREA_TOLERANCE,
    max_angle_tol=MAX_ANGLE_TOLERANCE,
    max_length_tol=MAX_LENGTH_TOLERANCE,
)

### 3.3. Generate interfaces with strain matcher
Interfaces are sorted by size and strain.

In [ ]:
from mat3ra.made.tools.build.interface import ZSLStrainMatchingInterfaceBuilder, ZSLStrainMatchingInterfaceBuilderParameters

matched_interfaces_builder = ZSLStrainMatchingInterfaceBuilder(build_parameters=ZSLStrainMatchingInterfaceBuilderParameters(strain_matching_parameters=zsl_strain_matching_parameters))

interfaces_sorted_by_size_and_strain= matched_interfaces_builder.get_materials(configuration=interface_configuration)

### 3.4. Plot interfaces by size and strain


In [ ]:
from utils.plot import plot_strain_vs_atoms

PLOT_SETTINGS = {
    "HEIGHT": 600,
    "X_SCALE": "log",  # or linear
    "Y_SCALE": "log",  # or linear
}

plot_strain_vs_atoms(interfaces_sorted_by_size_and_strain, PLOT_SETTINGS)

### 3.5. Select the interface

Select the index for the interface with the lowest strain and the smallest number of atoms.

In [ ]:
# select the first interface with the lowest strain and the smallest number of atoms
interfaces_slice_range_or_index = slice(0, 1)
selected_interfaces = interfaces_sorted_by_size_and_strain[interfaces_slice_range_or_index]

## 4. Preview the selected material

In [ ]:
visualize(selected_interfaces, repetitions=[3, 3, 1])
visualize(selected_interfaces, repetitions=[3, 3, 1], rotation="-90x")

## 5. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials
set_materials(selected_interfaces)